<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Visualizaci%C3%B3n_Puntos_Fijos_Version_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MONITOREO INSTRUMENTACIÓN PRESAS

#**Carga Masiva de datos Crudos**

In [ ]:
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets

# Diccionarios para almacenar por tipo de archivo
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]

datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

# Función para detectar tipo de instrumento por nombre
def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return None  # Puntos fijos sin margen, no válido
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# --- Widget de carga de archivos ---
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'}
)

output = widgets.Output()

# Función principal de carga
def cargar_archivos(change):
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        for nombre_archivo, archivo_info in archivos.items():
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)

                if not instrumento:
                    print(f"❌ Instrumento no reconocido o mal nombrado: {nombre_archivo}")
                    continue

                # Cargar el archivo
                if extension == 'csv':
                    df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                    datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (CSV)")
                elif extension == 'xlsx':
                    df = pd.read_excel(io.BytesIO(contenido))
                    datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (XLSX)")
                else:
                    print(f"❌ Formato no compatible: {nombre_archivo}")
            except Exception as e:
                print(f"❌ Error al procesar {nombre_archivo}: {e}")

        mostrar_menu()

# Función de visualización dinámica
def mostrar_menu():
    opciones = []
    for origen in ['csv', 'xlsx']:
        for instrumento in instrumentos:
            opciones.append(f"{instrumento} ({origen})")

    selector = widgets.Dropdown(
        options=opciones,
        description='Seleccionar DataFrame:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='60%')
    )

    def mostrar_datos(change):
        clear_output(wait=True)
        display(upload_widget, output)
        seleccion = selector.value
        instrumento, origen = seleccion.split(" ")
        instrumento = instrumento.strip()
        origen = origen.strip("()")

        print(f"📊 Mostrando: {instrumento.upper()} ({origen.upper()})")
        if origen == "csv":
            display(datos_csv[instrumento].head())
        else:
            display(datos_xlsx[instrumento].head())

    selector.observe(mostrar_datos, names='value')
    display(selector)

# Conectar evento
upload_widget.observe(cargar_archivos, names='value')

# Mostrar interfaz
display(upload_widget)
display(output)

FileUpload(value={'AS175_20250608.csv': {'metadata': {'name': 'AS175_20250608.csv', 'type': 'text/csv', 'size'…

Output()

📊 Mostrando: EXTENSOMETRO (CSV)


,FECHA,EXTENSOMETRO,COTA_EXCAV._(MSNM),PROFUNDIDAD,Z/COTA_RELEV.,DIFERENCIAS_(MM),ACUMULADO_(MM),COTA_FINAL_EXCAV._(MSNM)
0,03/04/2022,EX-CH-1a,120.0,0.0,120.0,0.0,0.0,120.0
1,12/05/2022,EX-CH-1a,120.0,0.0,120.0,0.0,0.0,120.0
2,30/05/2022,EX-CH-1a,120.0,0.0,120.0,0.0,0.0,120.0
3,11/06/2022,EX-CH-1a,120.0,0.0,120.0,0.0,0.0,120.0
4,04/07/2022,EX-CH-1a,120.0,0.0,120.0,0.0,0.0,120.0


# 📌 Visualización Interactiva de Puntos Fijos – MI y MD - Origen de datos: CSV

Este script permite **visualizar de forma interactiva** los datos relevados por los **puntos fijos** de una represa en ambas márgenes:  
- **Margen Izquierda (MI)**
- **Margen Derecha (MD)**

## 🎯 Objetivo

Brindar una herramienta visual para:

- Explorar y comparar variables medidas por cada punto fijo.
- Filtrar por margen, instrumento, variable y año.
- Configurar el estilo del gráfico: líneas, puntos, áreas, etc.
- Personalizar la visualización: tamaño, paleta de colores, grosor de línea.
- Exportar las gráficas en distintos formatos de alta calidad.

Es ideal para técnicos, analistas o ingenieros que trabajan con datos de auscultación estructural, facilitando la interpretación visual y generación de informes gráficos profesionales.

---

## ⚙️ Dependencias necesarias

Para que el script funcione correctamente, asegurate de tener instaladas las siguientes bibliotecas:

!pip install plotly ipywidgets pandas kaleido

In [ ]:
%pip install ipywidgets plotly
from google.colab import output as colab_output
colab_output.enable_custom_widget_manager()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.6 MB/s eta 0:00:00


In [ ]:
%pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.3 MB/s eta 0:00:00


In [ ]:
!pip install ipywidgets

In [ ]:
# === VISUALIZACIÓN INTERACTIVA UNIFICADA PARA PUNTOS FIJOS (MI y MD) desde datos_csv ===

import pandas as pd

try:
    import plotly.graph_objects as go
    import plotly.colors as pc
    import ipywidgets as widgets
    from IPython.display import display, clear_output, HTML
    import importlib.util
    import os
except ImportError:
    raise ImportError("Ejecuta: pip install plotly ipywidgets")

# ---- USAR DATAFRAMES YA CARGADOS EN EL SCRIPT DE CARGA ----
try:
    df_mi = datos_csv["puntos_fijos_mi"]
    df_md = datos_csv["puntos_fijos_md"]
except Exception as e:
    print("❌ Error accediendo a los datos de puntos fijos desde 'datos_csv':", e)
else:
    datasets = {
        nombre: df.copy()
        for nombre, df in {
            "Margen Izquierda (MI)": df_mi,
            "Margen Derecha (MD)": df_md
        }.items()
        if not df.empty
    }

    if not datasets:
        print("⚠️ No hay datos disponibles en ninguna de las márgenes.")
    else:
        estilos_grafico = [
            "Curvas suaves (spline)", "Líneas rectas", "Puntos",
            "Líneas + Puntos", "Área apilada",
            "Área + Líneas", "Área + Líneas + Puntos"
        ]

        tamanios_imagen = {
            "Pequeño (600x400)": (600, 400),
            "Mediano (900x500)": (900, 500),
            "Grande (1200x700)": (1200, 700),
            "Extra grande (1600x1000)": (1600, 1000)
        }

        grosores = {
            "Fino (1px)": 1, "Normal (2px)": 2, "Medio (4px)": 4,
            "Grueso (7px)": 7, "Extra grueso (10px)": 10
        }

        paletas = {
            "Plotly": pc.qualitative.Plotly,
            "D3": pc.qualitative.D3,
            "Viridis": pc.sequential.Viridis,
            "Cividis": pc.sequential.Cividis,
            "Inferno": pc.sequential.Inferno,
            "Pastel": pc.qualitative.Pastel,
            "Bold": pc.qualitative.Bold,
            "Set1": pc.qualitative.Set1,
            "Dark2": pc.qualitative.Dark2
        }

        def actualizar_opciones(change=None):
            margen_seleccionado = margen_dropdown.value
            df_actual = datasets[margen_seleccionado]
            df_actual['FECHA'] = pd.to_datetime(df_actual['FECHA'], dayfirst=True, errors='coerce')

            columnas_excluidas = ['FECHA', 'INSTRUMENTO', 'MARGEN']
            variables = [col for col in df_actual.select_dtypes(include='number').columns if col not in columnas_excluidas]
            variable_dropdown.options = variables
            if variables:
                variable_dropdown.value = variables[0]

            puntos_fijos = sorted(df_actual['INSTRUMENTO'].dropna().unique())
            punto_dropdown.options = ["Todos"] + list(puntos_fijos)
            punto_dropdown.value = "Todos"

            anios = sorted(df_actual['FECHA'].dt.year.dropna().unique())
            anio_dropdown.options = ["Todos"] + [str(a) for a in anios]
            anio_dropdown.value = "Todos"

        margen_dropdown = widgets.Dropdown(options=list(datasets.keys()), value=list(datasets.keys())[0], description="Margen:")
        punto_dropdown = widgets.Dropdown(options=["Todos"], value="Todos", description="Punto Fijo:")
        variable_dropdown = widgets.Dropdown(options=[], description="Variable:")
        estilo_dropdown = widgets.Dropdown(options=estilos_grafico, value="Curvas suaves (spline)", description="Estilo gráfica:")
        anio_dropdown = widgets.Dropdown(options=["Todos"], value="Todos", description="Año:")
        tamanio_dropdown = widgets.Dropdown(options=list(tamanios_imagen.keys()), value="Mediano (900x500)", description="Tamaño:")
        grosor_dropdown = widgets.Dropdown(options=list(grosores.keys()), value="Normal (2px)", description="Grosor línea:")
        paleta_dropdown = widgets.Dropdown(options=list(paletas.keys()), value="Plotly", description="Paleta colores:")
        boton = widgets.Button(description="Graficar", button_style="success")
        output = widgets.Output()
        output_guardar = widgets.Output()

        margen_dropdown.observe(actualizar_opciones, names='value')
        actualizar_opciones()

        formatos = {
            "PNG": ".png", "JPEG": ".jpg", "SVG": ".svg", "PDF": ".pdf", "HTML": ".html"
        }

        formato_dropdown = widgets.Dropdown(options=list(formatos.keys()), value="PNG", description="Formato:")
        ruta_text = widgets.Text(value="grafica_exportada", description="Ruta y nombre:")
        boton_guardar = widgets.Button(description="Guardar gráfica", button_style="info")

        def guardar_grafica(b=None):
            with output_guardar:
                clear_output(wait=True)
                ext = formatos[formato_dropdown.value]
                ruta_archivo = ruta_text.value
                if not ruta_archivo.lower().endswith(ext):
                    ruta_archivo += ext
                if 'fig' not in globals() or not isinstance(fig, go.Figure):
                    print("❌ Primero debes generar una gráfica.")
                    return
                try:
                    if formato_dropdown.value in ["PNG", "JPEG", "SVG", "PDF"]:
                        if importlib.util.find_spec("kaleido") is None:
                            print("❌ Para guardar como imagen/vector/pdf, instala 'kaleido':\n%pip install -U kaleido")
                            return
                        fig.write_image(ruta_archivo, format=formato_dropdown.value.lower())
                    elif formato_dropdown.value == "HTML":
                        fig.write_html(ruta_archivo)
                    else:
                        print("❌ Tipo de archivo no soportado.")
                        return
                except Exception as e:
                    print("❌ Error al guardar la gráfica:", e)
                    return

                print(f"✅ Gráfica guardada en: {os.path.abspath(ruta_archivo)}")
                if os.path.exists(ruta_archivo):
                    if 'google.colab' in str(get_ipython()):
                        from google.colab import files
                        files.download(ruta_archivo)
                    else:
                        ruta_abs = os.path.abspath(ruta_archivo)
                        display(HTML(f'<a href="file://{ruta_abs}" target="_blank">Descargar archivo</a>'))

        controles_guardar = widgets.HBox([formato_dropdown, ruta_text])

        def graficar(b=None):
            global fig
            with output:
                clear_output(wait=True)
                margen_seleccionado = margen_dropdown.value
                variable = variable_dropdown.value
                estilo = estilo_dropdown.value
                punto = punto_dropdown.value
                anio = anio_dropdown.value
                ancho, alto = tamanios_imagen[tamanio_dropdown.value]
                grosor = grosores[grosor_dropdown.value]
                paleta = paletas[paleta_dropdown.value]

                df = datasets[margen_seleccionado].copy()
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df_plot = df.dropna(subset=['FECHA', 'INSTRUMENTO', variable])

                if anio != "Todos":
                    df_plot = df_plot[df_plot['FECHA'].dt.year == int(anio)]
                if punto != "Todos":
                    df_plot = df_plot[df_plot['INSTRUMENTO'] == punto]

                if df_plot.empty:
                    print("No hay datos para graficar con la selección actual.")
                    return

                fig = go.Figure()
                instrumentos = sorted(df_plot['INSTRUMENTO'].unique())
                color_map = {pf: paleta[i % len(paleta)] for i, pf in enumerate(instrumentos)}

                for pf in instrumentos:
                    data_pf = df_plot[df_plot['INSTRUMENTO'] == pf]
                    line_args = dict(width=grosor, color=color_map[pf])
                    marker_args = dict(color=color_map[pf])

                    if estilo == "Curvas suaves (spline)":
                        fig.add_trace(go.Scatter(x=data_pf['FECHA'], y=data_pf[variable], mode="lines",
                                                 name=pf, line_shape="spline", line=line_args))
                    elif estilo == "Líneas rectas":
                        fig.add_trace(go.Scatter(x=data_pf['FECHA'], y=data_pf[variable], mode="lines",
                                                 name=pf, line_shape="linear", line=line_args))
                    elif estilo == "Puntos":
                        fig.add_trace(go.Scatter(x=data_pf['FECHA'], y=data_pf[variable], mode="markers",
                                                 name=pf, marker=marker_args))
                    elif estilo == "Líneas + Puntos":
                        fig.add_trace(go.Scatter(x=data_pf['FECHA'], y=data_pf[variable], mode="lines+markers",
                                                 name=pf, line_shape="linear", line=line_args, marker=marker_args))
                    elif estilo == "Área apilada":
                        fig.add_trace(go.Scatter(x=data_pf['FECHA'], y=data_pf[variable], mode="lines",
                                                 name=pf, stackgroup='one', line_shape="linear",
                                                 line=line_args, marker=marker_args))
                    elif estilo == "Área + Líneas":
                        fig.add_trace(go.Scatter(x=data_pf['FECHA'], y=data_pf[variable], mode="lines",
                                                 name=pf, fill="tozeroy", line_shape="linear",
                                                 line=line_args, marker=marker_args))
                    elif estilo == "Área + Líneas + Puntos":
                        fig.add_trace(go.Scatter(x=data_pf['FECHA'], y=data_pf[variable], mode="lines+markers",
                                                 name=pf, fill="tozeroy", line_shape="linear",
                                                 line=line_args, marker=marker_args))

                fig.update_layout(
                    width=ancho,
                    height=alto,
                    title=f"{margen_seleccionado}: {variable} en función del tiempo por Punto Fijo (PF)",
                    xaxis_title="Fecha",
                    yaxis_title=variable,
                    legend_title="INSTRUMENTO",
                    hovermode="x unified"
                )
                fig.show()

        # ---- UI ----
        display(HTML("<h2 style='color:#1866a3; margin-bottom: 5px'>Visualización Interactiva - Puntos Fijos (MI y MD)</h2>"))
        display(widgets.HBox([margen_dropdown, punto_dropdown, variable_dropdown, estilo_dropdown]))
        display(widgets.HBox([anio_dropdown, tamanio_dropdown, grosor_dropdown, paleta_dropdown]))
        display(boton)
        display(output)
        display(controles_guardar)
        display(boton_guardar)
        display(output_guardar)

        boton.on_click(graficar)
        boton_guardar.on_click(guardar_grafica)


Button(button_style='success', description='Graficar', style=ButtonStyle())

Output()

Button(button_style='info', description='Guardar gráfica', style=ButtonStyle())

Output()

# 📊 Visualización Interactiva de Puntos Fijos – MI y MD - Origen de datos: CSV y XLSX

Este notebook permite **explorar y graficar datos de instrumentación de puntos fijos** (MI y MD) de forma interactiva. Está diseñado para ser usado por técnicos, ingenieros y analistas que trabajan con datos de auscultación de presas o estructuras geotécnicas.

---

## 🎯 Objetivo del Script

Permite:
- Seleccionar la **fuente de datos** (`CSV` o `XLSX`) previamente cargada.
- Visualizar mediciones por **instrumento (PF)** y **margen**.
- Filtrar por **año**, **instrumento** o **variable** numérica.
- Cambiar el estilo del gráfico: líneas, puntos, áreas, curvas suaves, etc.
- Exportar la gráfica en distintos formatos (`PNG`, `PDF`, `SVG`, `HTML`, etc.).

---

## 🛠️ Requisitos

Instalá los siguientes paquetes para ejecutar el script correctamente:

```bash
pip install pandas plotly ipywidgets
pip install -U kaleido  # (solo si querés exportar como imagen o PDF)


In [ ]:
import pandas as pd

try:
    import plotly.graph_objects as go
    import plotly.colors as pc
    import ipywidgets as widgets
    from IPython.display import display, clear_output, HTML
    import importlib.util
    import os
except ImportError:
    raise ImportError("Ejecuta: pip install plotly ipywidgets")

# === Selectores y zonas persistentes ===
origen_dropdown = widgets.Dropdown(options=["CSV", "XLSX"], value="CSV", description="Origen:")
margen_dropdown = widgets.Dropdown(description="Margen:")
punto_dropdown = widgets.Dropdown(description="Punto Fijo:")
variable_dropdown = widgets.Dropdown(description="Variable:")
estilo_dropdown = widgets.Dropdown(
    options=["Curvas suaves (spline)", "Líneas rectas", "Puntos", "Líneas + Puntos",
             "Área apilada", "Área + Líneas", "Área + Líneas + Puntos"],
    value="Curvas suaves (spline)", description="Estilo gráfica:"
)
anio_dropdown = widgets.Dropdown(description="Año:")
tamanio_dropdown = widgets.Dropdown(
    options={"Pequeño (600x400)": (600, 400), "Mediano (900x500)": (900, 500),
             "Grande (1200x700)": (1200, 700), "Extra grande (1600x1000)": (1600, 1000)},
    value=(900, 500), description="Tamaño:"
)
grosor_dropdown = widgets.Dropdown(
    options={"Fino (1px)": 1, "Normal (2px)": 2, "Medio (4px)": 4, "Grueso (7px)": 7, "Extra grueso (10px)": 10},
    value=2, description="Grosor línea:"
)
paleta_dropdown = widgets.Dropdown(
    options={"Plotly": pc.qualitative.Plotly, "D3": pc.qualitative.D3,
             "Viridis": pc.sequential.Viridis, "Cividis": pc.sequential.Cividis,
             "Inferno": pc.sequential.Inferno, "Pastel": pc.qualitative.Pastel,
             "Bold": pc.qualitative.Bold, "Set1": pc.qualitative.Set1,
             "Dark2": pc.qualitative.Dark2},
    value=pc.qualitative.Plotly, description="Paleta colores:"
)
formato_dropdown = widgets.Dropdown(
    options={"PNG": ".png", "JPEG": ".jpg", "SVG": ".svg", "PDF": ".pdf", "HTML": ".html"},
    value=".png", description="Formato:"
)
ruta_text = widgets.Text(value="grafica_exportada", description="Ruta y nombre:")
boton = widgets.Button(description="Graficar", button_style="success")
boton_guardar = widgets.Button(description="Guardar gráfica", button_style="info")

# === Zonas dinámicas ===
output_grafica = widgets.Output()
output_mensaje = widgets.Output()

# === Función para cargar datasets según origen ===
def cargar_datasets():
    origen = origen_dropdown.value
    try:
        if origen == "CSV":
            df_mi = datos_csv["puntos_fijos_mi"]
            df_md = datos_csv["puntos_fijos_md"]
        else:
            df_mi = datos_xlsx["puntos_fijos_mi"]
            df_md = datos_xlsx["puntos_fijos_md"]

        datasets = {
            "Margen Izquierda (MI)": df_mi.copy(),
            "Margen Derecha (MD)": df_md.copy()
        }
        return {k: v for k, v in datasets.items() if not v.empty}
    except Exception as e:
        with output_mensaje:
            clear_output()
            print("❌ Error cargando datos:", e)
        return {}

# === Función para actualizar opciones según margen ===
def actualizar_opciones(*args):
    datasets = cargar_datasets()
    if not datasets:
        return

    margen = margen_dropdown.value or list(datasets.keys())[0]
    df = datasets[margen].copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')

    columnas_excluidas = ['FECHA', 'INSTRUMENTO', 'MARGEN']
    variables = [col for col in df.select_dtypes(include='number').columns if col not in columnas_excluidas]
    variable_dropdown.options = variables
    if variables:
        variable_dropdown.value = variables[0]

    punto_dropdown.options = ["Todos"] + sorted(df['INSTRUMENTO'].dropna().unique())
    anios = sorted(df['FECHA'].dt.year.dropna().unique())
    anio_dropdown.options = ["Todos"] + [str(a) for a in anios]

# === Función para graficar ===
def graficar(b=None):
    with output_grafica:
        clear_output()
    datasets = cargar_datasets()
    if not datasets:
        return

    df = datasets[margen_dropdown.value].copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    variable = variable_dropdown.value
    df = df.dropna(subset=['FECHA', 'INSTRUMENTO', variable])

    if anio_dropdown.value != "Todos":
        df = df[df['FECHA'].dt.year == int(anio_dropdown.value)]
    if punto_dropdown.value != "Todos":
        df = df[df['INSTRUMENTO'] == punto_dropdown.value]

    if df.empty:
        with output_mensaje:
            clear_output()
            print("⚠️ No hay datos para graficar.")
        return

    global fig
    fig = go.Figure()
    puntos = sorted(df['INSTRUMENTO'].unique())
    colores = {pf: paleta_dropdown.value[i % len(paleta_dropdown.value)] for i, pf in enumerate(puntos)}
    modo = {"Curvas suaves (spline)": "lines", "Líneas rectas": "lines", "Puntos": "markers",
            "Líneas + Puntos": "lines+markers", "Área apilada": "lines",
            "Área + Líneas": "lines", "Área + Líneas + Puntos": "lines+markers"}[estilo_dropdown.value]
    fill = "tozeroy" if "Área" in estilo_dropdown.value else None
    stack = 'one' if estilo_dropdown.value == "Área apilada" else None
    shape = "spline" if "spline" in estilo_dropdown.value else "linear"

    for pf in puntos:
        dpf = df[df['INSTRUMENTO'] == pf]
        fig.add_trace(go.Scatter(
            x=dpf['FECHA'], y=dpf[variable], mode=modo, name=pf,
            line=dict(width=grosor_dropdown.value, color=colores[pf]),
            marker=dict(color=colores[pf]), fill=fill, stackgroup=stack,
            line_shape=shape
        ))

    ancho, alto = tamanio_dropdown.value
    fig.update_layout(
        width=ancho, height=alto, title=f"{margen_dropdown.value}: {variable} en el tiempo",
        xaxis_title="Fecha", yaxis_title=variable, hovermode="x unified", legend_title="INSTRUMENTO"
    )
    with output_grafica:
        fig.show()

# === Función para guardar gráfica ===
def guardar_grafica(b=None):
    with output_mensaje:
        clear_output()
    ext = formato_dropdown.label
    ruta = ruta_text.value
    if not ruta.endswith(ext):
        ruta += ext
    if 'fig' not in globals():
        print("❌ Primero debes generar una gráfica.")
        return
    try:
        if formato_dropdown.value in [".png", ".jpg", ".svg", ".pdf"]:
            if importlib.util.find_spec("kaleido") is None:
                print("❌ Instala 'kaleido': %pip install -U kaleido")
                return
            fig.write_image(ruta)
        elif formato_dropdown.value == ".html":
            fig.write_html(ruta)
        print("✅ Gráfica guardada:", os.path.abspath(ruta))
    except Exception as e:
        print("❌ Error al guardar:", e)

# === Observadores ===
origen_dropdown.observe(lambda change: actualizar_todo(), names='value')
margen_dropdown.observe(actualizar_opciones, names='value')
boton.on_click(graficar)
boton_guardar.on_click(guardar_grafica)

def actualizar_todo():
    datasets = cargar_datasets()
    if datasets:
        margen_dropdown.options = list(datasets.keys())
        margen_dropdown.value = list(datasets.keys())[0]
        actualizar_opciones()

# === Mostrar interfaz ===
display(HTML("<h3 style='color:#1866a3'>Visualización Interactiva - Puntos Fijos</h3>"))
display(widgets.HBox([origen_dropdown, margen_dropdown, punto_dropdown, variable_dropdown]))
display(widgets.HBox([estilo_dropdown, anio_dropdown, tamanio_dropdown, grosor_dropdown, paleta_dropdown]))
display(boton)
display(widgets.HBox([formato_dropdown, ruta_text, boton_guardar]))
display(output_grafica)
display(output_mensaje)

# === Ejecutar inicial ===
actualizar_todo()


Button(button_style='success', description='Graficar', style=ButtonStyle())

Output()

Output()